In [239]:
from google.cloud import bigquery
client = bigquery.Client()
import pandas as pd
#import pandas_gbq
import numpy as np
import requests
import datetime as dt
import re
import json
from datetime import date, timedelta
from datetime import datetime
pd.set_option('display.width', 1000)
pd.set_option("max_colwidth",10000)
pd.set_option("max_rows",1000)
pd.set_option("max_columns",100)

In [240]:
current_date = datetime.today()
dias_atras = 0
#dias_atras = int(dias_atras)
str_day = (current_date - timedelta(days = dias_atras)).strftime("%Y-%m-%d")
print(str_day)

2024-04-09


In [241]:
df_logtable = pd.DataFrame()
sourceN = 0

### READ DATA SOURCES

In [242]:
table_name ='Worklogs'
query = """
 SELECT *
 FROM `saas-analytics-io.jira.%s`
 WHERE date(UPDATED) between '2022-01-01' and '%s'
       """%(table_name,str_day)

query_job = client.query(
  query,
    # Location must match that of the dataset(s) referenced in the query.
  location = "US",
)  # API request - starts the query
#print(query)

try:
    df_worklog = query_job.to_dataframe()
    #print(df_worklog.info())
    #print(df_worklog.shape)

    df_logtable.loc[sourceN,'date'] = str_day
    df_logtable.loc[sourceN,'tableName'] = table_name
    df_logtable.loc[sourceN,'tableRows'] = len(df_worklog.index)
    df_logtable.loc[sourceN,'tableColumns'] = len(df_worklog.columns)
    df_logtable.loc[sourceN,'status'] = 'ok'
    sourceN = sourceN+1
except Exception as e:
    df_worklog = query_job.to_dataframe()
    df_logtable.loc[sourceN,'date'] = str_day
    df_logtable.loc[sourceN,'tableName'] = table_name
    df_logtable.loc[sourceN,'tableRows'] = 0
    df_logtable.loc[sourceN,'tableColumns'] = 0
    df_logtable.loc[sourceN,'status'] = e

In [243]:
table_name ='Issues'
query = """
 SELECT *
 FROM `saas-analytics-io.jira.%s` 
 WHERE date(CREATED) between '2020-01-01' and '%s' """%(table_name,str_day)

query_job = client.query(
  query,
    # Location must match that of the dataset(s) referenced in the query.
  location = "US",
)  # API request - starts the query

try:
   # print(query)
    df_issues = query_job.to_dataframe()
    #print(df_issues.info())
    df_logtable.loc[sourceN,'date'] = str_day
    df_logtable.loc[sourceN,'tableName'] = table_name
    df_logtable.loc[sourceN,'tableRows'] = len(df_issues.index)
    df_logtable.loc[sourceN,'tableColumns'] = len(df_issues.columns)
    df_logtable.loc[sourceN,'status'] = 'ok'
    sourceN = sourceN+1
except Exception as e:
    df_issues = query_job.to_dataframe()
    df_logtable.loc[sourceN,'date'] = str_day
    df_logtable.loc[sourceN,'tableName'] = table_name
    df_logtable.loc[sourceN,'tableRows'] = 0
    df_logtable.loc[sourceN,'tableColumns'] = 0
    df_logtable.loc[sourceN,'status'] = e   


In [244]:
table_name ='Customers_12132'

query = """
 SELECT *
 FROM `saas-analytics-io.jira.%s` 
"""%(table_name)
query_job = client.query(
  query,
    # Location must match that of the dataset(s) referenced in the query.
  location = "US",
)  # API request - starts the query
#print(query)

try:
    df_customers = query_job.to_dataframe()
    #print(df_worklog.info())
    #print(df_worklog.shape)

    df_logtable.loc[sourceN,'date'] = str_day
    df_logtable.loc[sourceN,'tableName'] = table_name
    df_logtable.loc[sourceN,'tableRows'] = len(df_customers.index)
    df_logtable.loc[sourceN,'tableColumns'] = len(df_customers.columns)
    df_logtable.loc[sourceN,'status'] = 'ok'
    sourceN = sourceN+1
except Exception as e:
    df_customers = query_job.to_dataframe()
    df_logtable.loc[sourceN,'date'] = str_day
    df_logtable.loc[sourceN,'tableName'] = table_name
    df_logtable.loc[sourceN,'tableRows'] = 0
    df_logtable.loc[sourceN,'tableColumns'] = 0
    df_logtable.loc[sourceN,'status'] = e

In [245]:
df_issue_ticket = df_issues[df_issues['ISSUE_KEY'].str.startswith('TICKET')]

In [246]:
df_issues[df_issues['ISSUE_KEY'].str.startswith('TICKET-10822')]

,ISSUE_ID,ISSUE_KEY,ISSUE_TYPE_ID,ISSUE_TYPE_NAME,ISSUE_STATUS_ID,ISSUE_STATUS_NAME,SUMMARY,DESCRIPTION,PRIORITY,WATCHERS,WORK_RATIO,VOTES,RESOLUTION,PROJECT_ID,PROJECT_KEY,CURRENT_ASSIGNEE_ACCOUNT_ID,CURRENT_ASSIGNEE_NAME,CREATOR_ACCOUNT_ID,CREATOR_NAME,REPORTER_ACCOUNT_ID,REPORTER_NAME,ENVIRONMENT,CREATED,UPDATED,DUE_DATE,RESOLUTION_DATE,LAST_VIEWED,SECURITY_LEVEL_NAME,STATUS_CATEGORY_CHANGE_DATE,PARENT_ISSUE_ID,PARENT_ISSUE_KEY,PARENT_ISSUE_TYPE_ID,PARENT_ISSUE_TYPE_NAME,PARENT_PRIORITY,PARENT_ISSUE_STATUS_ID,PARENT_ISSUE_STATUS_NAME,Category_12050,Customer_11933,Customer_Category_12187,Customer_Name_11200,Customer_Priority_11879,Customer_Type_11954,Estimate_Sprints_12213,Platform_Version_12025,Progress___11891,Requesting_Customer_12273,Severity_11936,Severity_Level_11918,Story_Points_10400,Story_Points__Estimate__12242,Support_Level_11903,Tenant_12026,Tier_11886,__Effort_12125
42392,127315,TICKET-10822,10404,Incident,11302,Waiting for customer,[FALA] - Reader settings are not being applied in the Reader when the operator make inventory,"Hello team ! \nToday we had a troubleshoting session for one ticket were we identify the following ticket:\nWe changed the settings:\nQMulti Scan\t\nSession Multi Scan\t\nBut we saw that those changes were not applied in the reader , in order to improve the experience of the client in inventory we require to have those settings configured.\nRegards",None,5,-1,<NA>,None,10600,TICKET,60b0a1a0196bee00692d66e8,Cecilia Callejas,557058:ad134644-ab78-4775-a9a0-051e470e4baa,Gianina Teran,557058:ad134644-ab78-4775-a9a0-051e470e4baa,Gianina Teran,None,2024-03-15 20:14:28+00:00,2024-04-03 22:45:21+00:00,NaT,NaT,2024-03-15 20:45:39+00:00,None,2024-04-03 22:45:21+00:00,<NA>,None,<NA>,None,None,<NA>,None,None,None,None,None,None,Retail,NaN,10.01 Flash Release,0.0,None,None,S3,NaN,NaN,Gold,YT1,T1,NaN


In [247]:
df_issue_ticket_s = df_issue_ticket[['ISSUE_ID','ISSUE_KEY','ISSUE_TYPE_NAME','ISSUE_STATUS_NAME','SUMMARY',
                        'PRIORITY','PROJECT_KEY','CURRENT_ASSIGNEE_NAME','CREATOR_NAME','REPORTER_NAME',
                       'ENVIRONMENT','CREATED','RESOLUTION','RESOLUTION_DATE','SECURITY_LEVEL_NAME','STATUS_CATEGORY_CHANGE_DATE',
                       'Platform_Version_12025','Progress___11891','Severity_Level_11918','Support_Level_11903','Tenant_12026','Tier_11886','Customer_Type_11954']]

In [248]:
df_issue_ticket_s.head()

,ISSUE_ID,ISSUE_KEY,ISSUE_TYPE_NAME,ISSUE_STATUS_NAME,SUMMARY,PRIORITY,PROJECT_KEY,CURRENT_ASSIGNEE_NAME,CREATOR_NAME,REPORTER_NAME,ENVIRONMENT,CREATED,RESOLUTION,RESOLUTION_DATE,SECURITY_LEVEL_NAME,STATUS_CATEGORY_CHANGE_DATE,Platform_Version_12025,Progress___11891,Severity_Level_11918,Support_Level_11903,Tenant_12026,Tier_11886,Customer_Type_11954
44,78532,TICKET-6450,Question,Closed,RED - Question about hub command,None,TICKET,Amandine Aquino,Benoit MARTINETTI (EXT),Benoit MARTINETTI (EXT),None,2020-12-09 13:06:15+00:00,Done,2020-12-09 15:34:17+00:00,None,2020-12-09 15:34:17+00:00,None,0.0,S4,Gold,None,None,Retail
45,78533,TICKET-6451,Question,Closed,Sesame - PROBLEMA PISTOLA RFID PER E-commerce,None,TICKET,Miguel Callejas,support@mojix.com,Cedric LARRODE,None,2020-12-09 15:53:37+00:00,Done,2021-02-15 16:06:09+00:00,None,2021-02-15 16:06:09+00:00,None,0.0,S3,Gold,None,None,Retail
46,78609,TICKET-6465,Question,Closed,Eclipse - Question on webapp Eclipse.vuitton.biz,None,TICKET,Amandine Aquino,support@mojix.com,Uchso KOURK,None,2020-12-15 08:24:38+00:00,Done,2020-12-16 15:19:46+00:00,None,2020-12-16 15:19:46+00:00,None,0.0,S4,Gold,None,None,Retail
47,78624,TICKET-6473,Question,Closed,[support] Mojix Mobile application question,None,TICKET,Amandine Aquino,support@mojix.com,Ricardo Macias,None,2020-12-15 20:57:55+00:00,Done,2020-12-16 12:59:23+00:00,None,2020-12-16 12:59:23+00:00,None,0.0,S3,Gold,None,T1,Retail
48,79050,TICKET-6548,RMA,Closed,FedEx - Vizix Services Unavailable,None,TICKET,Diego Neyrot,13798,13798,None,2021-01-14 13:26:12+00:00,Duplicate,2021-01-19 14:28:28+00:00,None,2021-01-19 14:28:28+00:00,None,0.0,S4,Gold,None,T6,Industry


In [249]:
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('RED'),'TENANT']='RED'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('Eclipse'),'TENANT']='ECLIPSE'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('ECLIPSE'),'TENANT']='ECLIPSE'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('SESAME'),'TENANT']='SESAME'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('Sesame'),'TENANT']='SESAME'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('FedEx'),'TENANT']='FEDEX'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('HONDA'),'TENANT']='HONDA'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('Honda'),'TENANT']='HONDA'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('Ikea'),'TENANT']='IKEA'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('IKEA'),'TENANT']='IKEA'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('Mieloo'),'TENANT']='MIELOO'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('FALA'),'TENANT']='FALA'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('FALAPE'),'TENANT']='FALAPE'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('FALACO'),'TENANT']='FALACO'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('PEI'),'TENANT']='PEI'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('Perry Ellis'),'TENANT']='PEI'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('GREY'),'TENANT']='GREY'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('Grey'),'TENANT']='GREY'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('CENT'),'TENANT']='CENT'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('PERN'),'TENANT']='PERN'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('FISIA'),'TENANT']='FISIA'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('Fisia'),'TENANT']='FISIA'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('TOSCA'),'TENANT']='TOSCA'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('Tosca'),'TENANT']='TOSCA'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('CHIP'),'TENANT']='CHIP'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('Chipotle'),'TENANT']='CHIP'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('Raytheon'),'TENANT']='RAYTHEON'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('TRR'),'TENANT']='TRR'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('RLO2'),'TENANT']='RLO'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('CELINE'),'TENANT']='CELINE'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('Celine'),'TENANT']='CELINE'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('TORRA'),'TENANT']='TORRA'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('IVONNE'),'TENANT']='IVONNE'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('STUDIOF'),'TENANT']='STUDIOF'
df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('Studio F'),'TENANT']='STUDIOF'


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [250]:
#df_issue_ticket_s.loc[df_issue_ticket_s['SUMMARY'].str.contains('FALACO'),'TENANT']='Eclipse'

In [251]:
df_issue_ticket_s[df_issue_ticket_s['TENANT'].isnull()].count()

ISSUE_ID                       1403
ISSUE_KEY                      1403
ISSUE_TYPE_NAME                1403
ISSUE_STATUS_NAME              1403
SUMMARY                        1403
PRIORITY                       1403
PROJECT_KEY                    1403
CURRENT_ASSIGNEE_NAME          1403
CREATOR_NAME                   1403
REPORTER_NAME                  1403
ENVIRONMENT                       0
CREATED                        1403
RESOLUTION                      736
RESOLUTION_DATE                 736
SECURITY_LEVEL_NAME               0
STATUS_CATEGORY_CHANGE_DATE    1403
Platform_Version_12025          214
Progress___11891                622
Severity_Level_11918           1403
Support_Level_11903            1403
Tenant_12026                    216
Tier_11886                      636
Customer_Type_11954             718
TENANT                            0
dtype: int64

In [252]:
df_worklog_ticket = df_worklog[df_worklog['ISSUE_KEY'].str.startswith('TICKET')]

In [253]:
df_worklog_ticket_enh=pd.merge(df_worklog_ticket,df_issue_ticket_s[['ISSUE_KEY','ISSUE_TYPE_NAME','ISSUE_STATUS_NAME','SUMMARY','TENANT','Severity_Level_11918','Support_Level_11903','Customer_Type_11954','Tier_11886']],how='left', on ='ISSUE_KEY')

In [254]:
df_worklog_ticket_enh.loc[:,'LOGGED_HOURS']=df_worklog_ticket_enh['LOGGED_TIME']/60/60

In [255]:
df_worklog_ticket_enh.head()

,WORKLOG_ID,AUTHOR_ACCOUNT_ID,AUTHOR_NAME,UPDATE_ACCOUNT_ID,UPDATE_NAME,DESCRIPTION,START_DATE,LOGGED_TIME,CREATED,UPDATED,TEMPO_BILLABLE_TIME,ISSUE_ID,ISSUE_KEY,ISSUE_TYPE_NAME,ISSUE_STATUS_NAME,SUMMARY,TENANT,Severity_Level_11918,Support_Level_11903,Customer_Type_11954,Tier_11886,LOGGED_HOURS
0,62339,557058:0ed1278b-1fc5-4d4b-8db6-c45cca2c2497,Hugo Loza,557058:0ed1278b-1fc5-4d4b-8db6-c45cca2c2497,Hugo Loza,None,2022-04-05 11:56:16+00:00,9000,2022-04-05 14:26:25+00:00,2022-04-05 14:26:25+00:00,<NA>,88566,TICKET-7954,Incident,Closed,Perry Ellis - EPCs not changing status,PEI,S3,Gold,Retail,T1,2.5
1,64325,557058:0ed1278b-1fc5-4d4b-8db6-c45cca2c2497,Hugo Loza,557058:0ed1278b-1fc5-4d4b-8db6-c45cca2c2497,Hugo Loza,None,2022-07-29 21:15:30+00:00,5640,2022-07-29 22:49:57+00:00,2022-07-29 22:49:57+00:00,<NA>,94308,TICKET-8716,Incident,Closed,GREY - Erreur 4 sur poste Maro 18,GREY,S3,Gold,Retail,T3,1.566667
2,66253,557058:0ed1278b-1fc5-4d4b-8db6-c45cca2c2497,Hugo Loza,557058:0ed1278b-1fc5-4d4b-8db6-c45cca2c2497,Hugo Loza,None,2022-12-02 23:42:44+00:00,900,2022-12-02 23:58:57+00:00,2022-12-02 23:58:57+00:00,<NA>,99186,TICKET-9217,Incident,Closed,[HONDA] - OAUTH - User is not able to login,HONDA,S3,Gold,Industry,T1,0.25
3,74209,557058:0ed1278b-1fc5-4d4b-8db6-c45cca2c2497,Hugo Loza,557058:0ed1278b-1fc5-4d4b-8db6-c45cca2c2497,Hugo Loza,None,2023-12-06 15:54:35+00:00,540,2023-12-06 18:56:07+00:00,2023-12-06 18:56:07+00:00,<NA>,122431,TICKET-10501,Incident,Closed,[GREY] - Interruption système GREY le 15/11/2023 de 6h00 à 7h40,GREY,S3,Gold,Retail,T1,0.15
4,74210,557058:0ed1278b-1fc5-4d4b-8db6-c45cca2c2497,Hugo Loza,557058:0ed1278b-1fc5-4d4b-8db6-c45cca2c2497,Hugo Loza,None,2023-11-23 16:13:58+00:00,3180,2023-12-06 18:56:07+00:00,2023-12-06 18:56:07+00:00,<NA>,122792,TICKET-10520,Incident,Closed,[FALA] - EPC is not being read in inventory despite the gtin is valid and created in PF,FALA,S3,Gold,Retail,T1,0.883333


In [256]:
table = "saas-analytics-io.processed.jira_TICKET"
df_issue_ticket_s.to_gbq(table, if_exists='replace')
table = "saas-analytics-io.processed.jira_TICKET_worklog"
df_worklog_ticket_enh.to_gbq(table, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 10538.45it/s]
